In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip uninstall transformers -Y
    %pip install transformers

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/mres/model_harvesting


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -Y
Mounted at /content/gdrive
/content/gdrive/MyDrive/mres/model_harvesting


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
from IPython.display import clear_output

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def free_mem(vars):
    for v in vars: del v
    gc.collect()
    t.cuda.empty_cache()

In [4]:
models = {
    "llama2": "meta-llama/Llama-2-7b-chat-hf",
    "llama3": "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistral": "mistralai/Mistral-7B-v0.1"
}

model_name = "mistral"
model = AutoModelForCausalLM.from_pretrained(models[model_name], torch_dtype=t.float16).to(device); model.eval()
tokenizer = AutoTokenizer.from_pretrained(models[model_name])

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
tk_scores = [tokenizer.encode(str(i), add_special_tokens=False)[0] for i in range(1, 6)]
aspects = ["coherence", "consistency", "fluency", "relevance"]

### newsroom

In [6]:
prompts = pd.read_json("prompts/newsroom-theirs-score.jsonl", orient="records", lines=True)
for aspect in aspects:
    scores = []
    for i in trange(len(prompts), desc=aspect):
        prompt = prompts.at[prompts.index[i], f"prompt-{aspect}"]
        tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
        with t.no_grad(): out = model(tks)
        logits = out["logits"][:, -1, tk_scores]
        score = t.argmax(logits[0]).item() + 1
        scores.append(score)
        free_mem([tks, out, logits])
    prompts["score"] = scores
    prompts.to_json(f"{model_name}/scores/newsroom_{aspect}.jsonl", orient="records", lines=True)

coherence:   0%|          | 0/420 [00:00<?, ?it/s]

consistency:   0%|          | 0/420 [00:00<?, ?it/s]

fluency:   0%|          | 0/420 [00:00<?, ?it/s]

relevance:   0%|          | 0/420 [00:00<?, ?it/s]

### summeval

In [7]:
prompts = pd.read_json("prompts/summeval-theirs-score.jsonl", orient="records", lines=True)
for aspect in aspects:
    scores = []
    for i in trange(len(prompts), desc=aspect):
        prompt = prompts.at[prompts.index[i], f"prompt-{aspect}"]
        tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
        with t.no_grad(): out = model(tks)
        logits = out["logits"][:, -1, tk_scores]
        score = t.argmax(logits[0]).item() + 1
        scores.append(score)
        free_mem([tks, out, logits])
    prompts["score"] = scores
    prompts.to_json(f"{model_name}/scores/summeval_{aspect}.jsonl", orient="records", lines=True)

coherence:   0%|          | 0/1600 [00:00<?, ?it/s]

consistency:   0%|          | 0/1600 [00:00<?, ?it/s]

fluency:   0%|          | 0/1600 [00:00<?, ?it/s]

relevance:   0%|          | 0/1600 [00:00<?, ?it/s]